In [1]:
import os
import logging
import json
import uuid
from datetime import datetime
import time
import sys
import warnings
import pandas as pd
from tqdm import trange
from dotenv import load_dotenv
from io import BytesIO
import io
import zipfile
import re

# Adobe PDF Services imports
from adobe.pdfservices.operation.auth.service_principal_credentials import ServicePrincipalCredentials
from adobe.pdfservices.operation.exception.exceptions import ServiceApiException, ServiceUsageException, SdkException
from adobe.pdfservices.operation.io.cloud_asset import CloudAsset
from adobe.pdfservices.operation.io.stream_asset import StreamAsset
from adobe.pdfservices.operation.pdf_services import PDFServices
from adobe.pdfservices.operation.pdf_services_media_type import PDFServicesMediaType
from adobe.pdfservices.operation.pdfjobs.jobs.extract_pdf_job import ExtractPDFJob
from adobe.pdfservices.operation.pdfjobs.params.extract_pdf.extract_element_type import ExtractElementType
from adobe.pdfservices.operation.pdfjobs.params.extract_pdf.extract_pdf_params import ExtractPDFParams
from adobe.pdfservices.operation.pdfjobs.result.extract_pdf_result import ExtractPDFResult

# Pinecone and Langchain imports
from pinecone import Pinecone, ServerlessSpec
from pinecone_text.sparse import BM25Encoder
from langchain_groq import ChatGroq
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
warnings.filterwarnings("ignore")

# Load environment variables
load_dotenv()
groq_api_key = os.getenv('GROQ_API_KEY')
hf_key = os.getenv('HUGGINGFACE_API_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')
# openai_api_key = os.getenv('OPENAI_API_KEY')
dense_embedder_api = os.getenv("HF_API_URL")

# Define model
chat_model = "llama3-8b-8192"

from typing import Optional, Union, TypeAlias

# Import other necessary modules
from llama_index.legacy import Document
from llama_index.legacy import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.legacy.node_parser import SentenceWindowNodeParser, SemanticSplitterNodeParser
from llama_index.legacy.postprocessor import MetadataReplacementPostProcessor, SentenceTransformerRerank
from llama_index.legacy.schema import MetadataMode
from llama_index.legacy.ingestion import IngestionPipeline
from llama_index.legacy.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.legacy.vector_stores import PineconeVectorStore

In [2]:
# Initialize the logger
logging.basicConfig(level=logging.INFO)

class ExtractTextTableInfoFromPDF:
    def __init__(self, file_path):
        self.unique_id = str(uuid.uuid4())
        try:
            file = open(file_path, 'rb')
            input_stream = file.read()
            file.close()

            # Initial setup, create credentials instance
            credentials = ServicePrincipalCredentials(
                client_id=os.getenv('ADOBE_SERVICES_CLIENT_ID'),
                client_secret=os.getenv('ADOBE_SERVICES_CLIENT_SECRET')
            )

            # Creates a PDF Services instance
            pdf_services = PDFServices(credentials=credentials)

            # Creates an asset(s) from source file(s) and upload
            input_asset = pdf_services.upload(input_stream=input_stream, mime_type=PDFServicesMediaType.PDF)

            # Create parameters for the job
            extract_pdf_params = ExtractPDFParams(
                elements_to_extract=[ExtractElementType.TEXT, ExtractElementType.TABLES],
            )

            # Creates a new job instance
            extract_pdf_job = ExtractPDFJob(input_asset=input_asset, extract_pdf_params=extract_pdf_params)

            # Submit the job and gets the job result
            location = pdf_services.submit(extract_pdf_job)
            pdf_services_response = pdf_services.get_job_result(location, ExtractPDFResult)

            # Get content from the resulting asset(s)
            result_asset: CloudAsset = pdf_services_response.get_result().get_resource()
            stream_asset: StreamAsset = pdf_services.get_content(result_asset)
            
            zip_bytes = io.BytesIO(stream_asset.get_input_stream())
            with zipfile.ZipFile(zip_bytes, 'r') as zip_ref:
                # Extract all the contents into memory
                self.extracted_data = {name: zip_ref.read(name) for name in zip_ref.namelist()}
                
        except (ServiceApiException, ServiceUsageException, SdkException) as e:
            logging.exception(f'Exception encountered while executing operation: {e}')
    
    # Generates a string containing a directory structure and file name for the output file using unique_id
    @staticmethod
    def create_output_file_path(unique_id: str) -> str:
        os.makedirs("../data/Extracted_data", exist_ok=True)
        return f"../data/Extracted_data/{unique_id}.zip"

    @classmethod
    def create_with_unique_id(cls, file_path):
        instance = cls(file_path)
        return instance, instance.unique_id

In [3]:
# Get the extracted data from the extractor
def get_extracted_data(extracted_data):
    if 'structuredData.json' in extracted_data:
        json_data = json.loads(extracted_data['structuredData.json'])
    return json_data

# Function to initialise a flexible text splitter
def initialise_text_splitter(chunk_size, chunk_overlap):
    separator_list = ["\n\n", "\n", ". ", "!", "?", ",", " ", "", ")", "("]
    try:
        text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=chunk_size,
                chunk_overlap=chunk_overlap,
                length_function=len,
                separators=separator_list)
    except Exception as e:
        logging.error(f"Failed to initialize text splitter: {e}")
        return None
    return text_splitter

# Function to obtain text chunks using the text splitter
def get_text_chunks(file_path, json_data):
    if 'elements' not in json_data:
        logging.error("Missing 'elements' key in json_data")
        raise ValueError("Missing 'elements' key in json_data")
        
    file_name = file_path.split("/")[-1]

    # Chunks are split by pages here
    page_text = ""
    start_page = 0
    
    all_texts = []
    
    list_label = ""
    for i in range(len(json_data['elements'])):
        try:
            current_page = json_data['elements'][i]['Page']
        except KeyError:
            logging.warning(f"Missing 'Page' key in element at index {i}")
            continue

        try:
            if current_page > start_page:
                # Update the new page number
                start_page = current_page               
         
                all_texts.append({'ElementType': 'Text', 'file_name': file_name, 'Page': current_page, 'Text': page_text})
                page_text = ""
                list_label = ""
            else:
                if 'Text' in json_data['elements'][i]:  # Check if Text is not empty
                    if json_data['elements'][i]['Path'].endswith("Lbl") and not json_data['elements'][i]['Path'].startswith("//Document/Table"):
                        list_label = json_data['elements'][i]['Text']
                    else:
                        if list_label:
                            page_text += list_label + json_data['elements'][i]['Text']
                            list_label = ""  # Reset list label to empty string
                        else:
                            page_text += json_data['elements'][i]['Text'] + "\n"
        except KeyError as e:
            logging.warning(f"Key error in json_data['elements'][i] processing at index {i}: {e}")
    
    
    # Processing the last page of the text
    if page_text:
        all_texts.append({'ElementType': 'Text', 'file_name': file_name, 'Page': current_page, 'Text': page_text})

    return all_texts

# Function to derive the nodes from the text chunks
def convert_pagetexts_to_nodes(text_chunks):
    
    # Function to clean up the text in each node
    def clean_up_text(content: str) -> str:
        """
        Remove unwanted characters and patterns in text input.
        :param content: Text input.
        :return: Cleaned version of original text input.
        """

        # Fix hyphenated words broken by newline
        content = re.sub(r'(\w+)-\n(\w+)', r'\1\2', content)

        # Remove specific unwanted patterns and characters
        unwanted_patterns = [
            "\\n", "  —", "——————————", "—————————", "—————",
            r'\\u[\dA-Fa-f]{4}', r'\uf075', r'\uf0b7'
        ]
        for pattern in unwanted_patterns:
            content = re.sub(pattern, "", content)

        # Fix improperly spaced hyphenated words and normalize whitespace
        content = re.sub(r'(\w)\s*-\s*(\w)', r'\1-\2', content)
        content = re.sub(r'\s+', ' ', content)
        return content
    
    # Conversion of text chunks to Documents
    page_documents = [Document(text=chunk['Text'],
                            metadata={
                                "file_name": chunk['file_name'],
                                "page": chunk['Page']
                                },
                            excluded_llm_metadata_keys=["file_name"],
                            metadata_seperator="::",
                            metadata_template="{key}=>{value}",
                            text_template="Metadata: {metadata_str}\n-----\nContent: {content}")
                            
                            for chunk in text_chunks]

    # Clean the texts in each page
    page_nodes = []
    for d in page_documents:
        cleaned_text = clean_up_text(d.text)
        d.text = cleaned_text
        page_nodes.append(d)
    return page_nodes

# Function to get the unique tables from all the table elements
def extract_unique_tables(table_elements):
    tables = set()
    for item in table_elements:
        match = re.search(r'/Table(\[\d+\])?', item['Path'])
        if match:
            tables.add('Table' + (match.group(1) if match.group(1) else ''))
    
    unique_tables = list(tables)
    unique_tables[0] += "/"

    extracted_tables = {}
    i=0
    for table_name in unique_tables:
        table = []
        for el in table_elements:
            if table_name in el['Path']:
                # ADjust this here if u need to extract more information from the table elements
                table.append({"path": el['Path'], "text": el['Text'], "Page": el["Page"]})
        extracted_tables[i+1] = table
        i += 1

    return dict(sorted(extracted_tables.items()))

# Function to take in table elements from a specific table and convert it to a pandas dataframe
def transform_table(table, output_file_path):
    # Function to decide if table has 2 indexes
    def is_2_index(first_row):
        for el in first_row:
            if "TH/" in el['path']:
                return False
            else:
                return True
    
    # Function to decide if the table's row is the header   
    def is_row_header(first_row):
        for el in first_row:
            if "TH" not in el['path']:
                return False
        return True

    # Function to get the table dimensions
    def get_table_dim(table):
        # Regular expressions to find row and column indices
        row_regex = re.compile(r'/TR\[(\d+)\]')
        col_regex = re.compile(r'/TD\[(\d+)\]')

        max_row = 0
        max_col = 0

        for entry in table:
            path = entry['path']  # Extract path from the dictionary
            
            # Extract row and column indices
            row_match = row_regex.search(path)
            col_match = col_regex.search(path)
            
            # Update max row index
            if row_match:
                row_index = int(row_match.group(1))
                max_row = max(max_row, row_index)
            
            # Update max column index
            if col_match:
                col_index = int(col_match.group(1))
                max_col = max(max_col, col_index)
            elif '/TD/' in path and not col_match:  # Check for default first column
                max_col = max(max_col, 1)  # Ensure first column is counted
        return (max_row, max_col)    

    # Function to check if the API could not identify the index of the table
    def is_unidentified(table):
        table_dim = get_table_dim(table)
        min_headers = min(table_dim)
        header_count = len([el for el in table if "TH" in el['path']])
        if header_count < min_headers:
            return True
        return False

    # Function to transform the unidentified table by determining its index using LLM
    def transform_unidentified_table(table):
        # Function to get the first 3 rows 
        def get_first_three_rows(table):
                # Regular expression to extract row index
                row_regex = re.compile(r'/TR\[(\d+)\]')
                
                # List to hold the first three rows table
                rows = []
                
                for entry in table:
                    path = entry['path']
                    # Find row index
                    row_match = row_regex.search(path)
                    
                    if row_match:
                        row_index = int(row_match.group(1))
                        # Check if the row index is within the first three rows
                        if 1 <= row_index <= 3:
                            rows.append(entry)
                    elif '/TR/' in path:  # Check for paths that imply the first row implicitly
                        rows.append(entry)
            
                table_first_3_rows = [] # This will be a 2D list
                table_rows = set([re.search(r'Table(?:\[\d+\])?/TR(?:\[\d+\])?', item['path']).group() for item in table if re.search(r'Table(?:\[\d+\])?/TR(?:\[\d+\])?', item['path'])])
                uniq_rows = sorted([row_name.split("/")[-1] for row_name in list(table_rows)[:3]])
                uniq_rows[0] += "/"


                for i in range(len(uniq_rows)):
                    row = [el for el in table if uniq_rows[i] in el['path']]
                    unique_tds = set()
                    for item in row:
                                path_parts = item['path'].split('/')
                                for part in path_parts:
                                    if 'TD' in part:
                                        unique_tds.add(part)
                    # Convert the set to a list and sort it for consistent output
                    unique_tds_list = sorted(list(unique_tds))
                            
                    sections_of_row = []
                    for td in unique_tds_list:
                                td_section = ""
                                for i in range(len(row)):
                                    if i ==0:
                                        td += "/"
                                    if td in row[i]['path']:
                                        td_section += row[i]['text'].strip()

                                sections_of_row.append(td_section)
                    
                    # print(sections_of_row)
                    table_first_3_rows.append(sections_of_row)

                # account for case where there are 2 index columns:
                min_dim = max(list(map(lambda x: len(x), table_first_3_rows)))
                if len(table_first_3_rows[0]) < min_dim:
                    table_first_3_rows[0].insert(0, "empty")
                return table_first_3_rows

        # Function to get the first 3 rows of the table into a string to parse to the LLM 
        def get_table_check_string(table_first_3_rows):
            table_str = ''
            for i in range(len(table_first_3_rows)):
                table_str += f'''Row{i+1}: {" | ".join(table_first_3_rows[i])}\n'''
            return table_str

        # Function to clean the output if it decides to explain its choice.
        def clean_llama_output_if_string(input_str):
            """
            Extracts a dictionary-like substring from the given input string.
            
            Args:
            input_str (str): The input string containing the dictionary-like substring.
            
            Returns:
            dict: The extracted dictionary as a Python dictionary.
            """
            # Use a regular expression to find the dictionary-like substring
            match = re.search(r'\{.*?\}', input_str)
            
            if match:
                dict_str = match.group()
                try:
                    # Safely evaluate the dictionary string
                    extracted_dict = eval(dict_str)
                    if isinstance(extracted_dict, dict):
                        return extracted_dict
                except (SyntaxError, NameError):
                    pass
            
            return None

        # Function to decide the indexing of the table
        def eval_table_index_llama(table_str):
            class Header(BaseModel):
                index: int = Field(description=(
                    '''Index of the table indicating the type of indexing:\n\
                        0 - No indexes\n\
                        1 - Indexed by the first column\n\
                        2 - Indexed by the first row\n\
                        3 - Indexed by both the first row and the first column'''))

            parser = JsonOutputParser(pydantic_object=Header)

            chat = ChatGroq(temperature=0, model_name="llama3-8b-8192")
            
            template = '''You will assist me in deciding, based on the first 3 entries of a table, whether the first row or the first colum should be the header, or if both the first column and first row are headers
        Table:
        {table}

        Determine the indexing type of the table and output:
        - 0 if the table is indexed by the first column.
        - 1 if the table is indexed by the first row.
        - 2 if the table is indexed by both the first row and the first column.   
                    
        {format_instructions}'''
            prompt = PromptTemplate(
                template=template,
                input_variables=["table"],
                partial_variables={"format_instructions": parser.get_format_instructions()},
            )
                
            chain = prompt | chat | parser
            return chain.invoke({"table": table_str})
        
        table_first_3_rows = get_first_three_rows(table)
        table_str = get_table_check_string(table_first_3_rows)
        header_output = eval_table_index_llama(table_str)
        if isinstance(header_output, str):
            header_output = clean_llama_output_if_string(header_output)
        header_index = header_output['index']
        # Now need diff cases of how to handle the table if the model outputs, 0,1, or 2
            # where 0 is the first col, 1 is second col, 2 is for both cols are headers
        pass
    
    # If the table cannot be identified by the ExtractTable API
    if is_unidentified(table):
        processed_unidentified_table = transform_unidentified_table(table)
        pass
    
    # only need to look at the first row
    first_row = [el for el in table if "TR/" in el['path']]
    
    # If the table has 2 indexes
    if is_2_index(first_row):
            print("This table has 2 indexes")
            # Function to produce table which has 2 indexes
            def get_2_index_table(table):
                table_rows = set([re.search(r'Table(?:\[\d+\])?/TR(?:\[\d+\])?', item['path']).group() for item in table if re.search(r'Table(?:\[\d+\])?/TR(?:\[\d+\])?', item['path'])])
                uniq_rows = sorted([row_name.split("/")[-1] for row_name in list(table_rows)])

                # Get indexes from the first row:
                row_indexes = [el['text'].strip() for el in table if "TR/TH" in el['path']]

                data = {}

                # Only look at second row onwards
                for i in range(1,len(uniq_rows)):
                    row_name = uniq_rows[i]
                    row = [el for el in table if row_name in el['path']]
                    row_key = row[0]['text'].strip()
                    
                    unique_tds = set()
                    for item in row:
                        path_parts = item['path'].split('/')
                        for part in path_parts:
                            if 'TD' in part:
                                unique_tds.add(part)
                    # Convert the set to a list and sort it for consistent output
                    unique_tds_list = sorted(list(unique_tds))

                    sections_of_row = []
                    for td in unique_tds_list:
                        td_section = ""
                        for i in range(len(row)):
                            if i ==0:
                                td += "/"
                            if td in row[i]['path']:
                                td_section += row[i]['text'].strip()
                                
                        sections_of_row.append(td_section)
                    
                    data[row_key] = sections_of_row
                    
                df = pd.DataFrame(data, index=row_indexes).T
                df.to_csv(output_file_path)
                return df    
            
            df = get_2_index_table(table)
            return df
        
    # If the table only has one index
    else:
            print("This table has 1 index")

            # If the header for this df is the row
            if is_row_header(first_row):
                print("This is a row indexed table")
                def get_row_header_table(table):
                    table_rows = set([re.search(r'Table(?:\[\d+\])?/TR(?:\[\d+\])?', item['path']).group() for item in table if re.search(r'Table(?:\[\d+\])?/TR(?:\[\d+\])?', item['path'])])
                    uniq_rows = sorted([row_name.split("/")[-1] for row_name in list(table_rows)])

                    # Table headers, also the keys
                    headers = [el['text'].strip() for el in table if uniq_rows[0]+"/" in el['path']]
                    rows = []
                    for i in range(1,len(uniq_rows)):
                        row_name = uniq_rows[i]
                        row = [el for el in table if row_name in el['path']]
                        # rows.append(row)
                        
                        unique_tds = set()
                        for item in row:
                            path_parts = item['path'].split('/')
                            for part in path_parts:
                                if 'TD' in part:
                                    unique_tds.add(part)
                        # Convert the set to a list and sort it for consistent output
                        unique_tds_list = sorted(list(unique_tds))

                        sections_of_row = []
                        for td in unique_tds_list:
                            td_section = ""
                            for i in range(len(row)):
                                if i ==0:
                                    td += "/"
                                if td in row[i]['path']:
                                    td_section += row[i]['text'].strip()
                                    
                            sections_of_row.append(td_section)
                        rows.append(sections_of_row)
                                    
                    df = pd.DataFrame(rows, columns=headers)
                    return df
                
                df = get_row_header_table(table)
                df.to_csv(output_file_path, index=False)
                return df
                    
            # The header for this df is the column
            else:
                print("This table is a column indexed table...")
                def get_column_header_table(table):
                    table_rows = set([re.search(r'Table(?:\[\d+\])?/TR(?:\[\d+\])?', item['path']).group() for item in table if re.search(r'Table(?:\[\d+\])?/TR(?:\[\d+\])?', item['path'])])
                    uniq_rows = sorted([row_name.split("/")[-1] for row_name in list(table_rows)])

                    data = {}

                    for i in range(len(uniq_rows)):
                                        row_name = uniq_rows[i]
                                        if i == 0:
                                            row = [el for el in table if row_name+"/" in el['path']]
                                        else:
                                            row = [el for el in table if row_name in el['path']]
                                        
                                        unique_tds = set()
                                        for item in row:
                                            path_parts = item['path'].split('/')
                                            for part in path_parts:
                                                if 'TD' in part:
                                                    unique_tds.add(part)
                                        # Convert the set to a list and sort it for consistent output
                                        unique_tds_list = sorted(list(unique_tds))
                                        
                                        sections_of_row = []
                                        for td in unique_tds_list:
                                            td_section = ""
                                            for i in range(len(row)):
                                                if i ==0:
                                                    td += "/"
                                                if td in row[i]['path']:
                                                    td_section += row[i]['text'].strip()
                                                    
                                            sections_of_row.append(td_section)
                                                    
                                        row_key = row[0]['text'].strip()
                                        data[row_key] = sections_of_row
                    df = pd.DataFrame.from_dict(data, orient='index').T
                    return df
                                    
                df = get_column_header_table(table)
                df.to_csv(output_file_path, index=False)
                return df

# Function that saves the tables in CSV format to a unique job id       
def save_tables_to_csv(extracted_tables, table_output_directory):
    
    os.makedirs(table_output_directory, exist_ok=True)
    for table_num, table in extracted_tables.items():
        output_file_path = os.path.join(table_output_directory, f"table_{table_num}.csv")
        df = transform_table(table, output_file_path)


## Runing the script

In [4]:
file_path = '../PDF/Invest First Max Summary.pdf'
# extractor, unique_id = ExtractTextTableInfoFromPDF.create_with_unique_id(file_path)
# extracted_data = extractor.extracted_data
# pdf_data = get_extracted_data(extracted_data)
fname = file_path.split("/")[-1]
# with open(f"../data/{fname}.json", "w", encoding="utf-8") as fout:
    # json.dump(pdf_data, fout)

In [5]:
with open("../data/Invest First Max Summary.pdf.json", "r", encoding='utf-8') as fin:
    pdf_data = json.load(fin)
    
def extract_tables_from_pdf(pdf_data):
    table_elements = [el for el in pdf_data['elements'] if "Table" in el['Path'] and 'Text' in el and "TR" in el['Path']]
    print("\nUnique ID:", unique_id)

    # IF there are even any table elements in the PDF
    if table_elements:
        table_output_directory = f"../data/{unique_id}" 
        extracted_tables = extract_unique_tables(table_elements)    
        save_tables_to_csv(extracted_tables, table_output_directory)

# Section here to derive the csvs from the table elements
# Uncomment to extract tables (for now its not ready yet)
# extract_tables_from_pdf(pdf_data)

# Section here to derive the nodes from the text elements
# text_splitter = initialise_text_splitter(1000, 50)
page_texts = get_text_chunks(file_path, pdf_data)
page_nodes = convert_pagetexts_to_nodes(page_texts)

2024-07-05 22:42:05,019 - WARNING - Missing 'Page' key in element at index 217
2024-07-05 22:42:05,020 - WARNING - Missing 'Page' key in element at index 218
2024-07-05 22:42:05,020 - WARNING - Missing 'Page' key in element at index 219
2024-07-05 22:42:05,020 - WARNING - Missing 'Page' key in element at index 220
2024-07-05 22:42:05,021 - WARNING - Missing 'Page' key in element at index 221
2024-07-05 22:42:05,021 - WARNING - Missing 'Page' key in element at index 222
2024-07-05 22:42:05,021 - WARNING - Missing 'Page' key in element at index 223
2024-07-05 22:42:05,022 - WARNING - Missing 'Page' key in element at index 224
2024-07-05 22:42:05,022 - WARNING - Missing 'Page' key in element at index 225
2024-07-05 22:42:05,022 - WARNING - Missing 'Page' key in element at index 226
2024-07-05 22:42:05,023 - WARNING - Missing 'Page' key in element at index 227
2024-07-05 22:42:05,023 - WARNING - Missing 'Page' key in element at index 228
2024-07-05 22:42:05,023 - WARNING - Missing 'Page' k

## Set up ingestion pipeline to pipe data into pinecone

In [6]:
embed_model = HuggingFaceEmbedding(
    model_name='sentence-transformers/all-mpnet-base-v2')
if embed_model:
    logging.info("Embedding model loaded...")
embed_model

2024-07-05 22:42:11,963 - INFO - Embedding model loaded...


HuggingFaceEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.legacy.callbacks.base.CallbackManager object at 0x32a8dac20>, tokenizer_name='sentence-transformers/all-mpnet-base-v2', max_length=514, pooling=<Pooling.MEAN: 'mean'>, normalize=True, query_instruction=None, text_instruction=None, cache_folder=None)

In [7]:
# Define the initial pipeline
pipeline = IngestionPipeline(
    transformations=[
        SemanticSplitterNodeParser(
            buffer_size=1,
            breakpoint_percentile_threshold=95,
            embed_model=embed_model,  # Pass the dictionary instead of the raw instance
        ),
        embed_model,  # Use the model instance directly in the pipeline
    ],
)

In [8]:
# Initialize connection to Pinecone
pc = Pinecone(api_key=pinecone_api_key)
unique_id = "zac-notes"
index_name = unique_id

# Create your index (can skip this step if your index already exists)
hybrid_search=True
if hybrid_search:
    pc_similarity_metric = "dotproduct"
else:
    pc_similarity_metric = "cosine"

# pc.create_index(
#     index_name,
#     dimension=768,
#     metric=pc_similarity_metric,
#     spec=ServerlessSpec(cloud="aws", region="us-east-1")
# )

# Initialize your index
pinecone_index = pc.Index(index_name)

# Initialize VectorStore
vector_store = PineconeVectorStore(pinecone_index=pinecone_index, llm=None)

In [9]:
# Adding the vector store index into the insertion pipeline
pipeline = IngestionPipeline(
    transformations=[
        SemanticSplitterNodeParser(
            buffer_size=1,
            breakpoint_percentile_threshold=95,
            embed_model=embed_model,
            ),
        embed_model,
        ],
        vector_store=vector_store  # Our new addition
    )

# Pushing the first 3 pages of PDF to the pinecone index. 
first_3_pages = page_nodes[:3]
# pipeline.run(documents=first_3_pages)

In [10]:
def describe_pinecone_index():
    pc_stats = pinecone_index.describe_index_stats()
    print(f"Dimension of embeddings in this index: {pc_stats['dimension']}")
    print(f"Index fullness is at {pc_stats['index_fullness']}")
    print(f"Namespaces in this index: {pc_stats['namespaces']}")
    print(f"Total vector count at {pc_stats['total_vector_count']}")
    
describe_pinecone_index()

Dimension of embeddings in this index: 768
Index fullness is at 0.0
Namespaces in this index: {'': {'vector_count': 6}}
Total vector count at 6


## Different User Query Types:
User queries in an RAG application vary based on individual intent. For these diverse query types, it’s essential to fine-tune the Alpha parameter. This process involves routing each user query to a specific Alpha value for effective retrieval and response synthesis. Microsoft has identified various user query categories, and we have selected a few for tuning our hybrid search. Below are the different user query types we considered:

- Web Search Queries: Brief queries similar to those typically inputted into search engines.
- Concept Seeking Queries: Abstract questions that necessitate detailed, multi-sentence answers.
- Fact Seeking Queries: Queries that have a single, definitive answer.
- Keyword Queries: Concise queries composed solely of crucial identifier words.
- Queries With Misspellings: Queries containing typos, transpositions, and common misspellings.
- Exact Sub-string Searches: Queries that exactly match sub-strings from the original context.

Tuning the alpha value to adjust how much of vector and how much of semantic search we should be using:
- https://medium.com/llamaindex-blog/llamaindex-enhancing-retrieval-performance-with-alpha-tuning-in-hybrid-search-in-rag-135d0c9b8a00
  

In [33]:
from llama_index.legacy.retrievers import VectorIndexRetriever
# Instantiate VectorStoreIndex object from our vector_store object
import llama_index.legacy.llms

llm = ChatGroq(model="llama3-70b-8192", api_key=groq_api_key)

vector_index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

# Grab 5 search results
retriever = VectorIndexRetriever(index=vector_index, similarity_top_k=5)

# # Query vector DB
answer = retriever.retrieve('What is FWD Invest First max about?')
for ans in answer:
    print(ans)
    print("-" * 80)


NameError: name 'Groq' is not defined

## Continuation of table extraction (For unidentified tables)

In [24]:
# Ensure that the table here has the TH's replaced with TD's
table = extracted_tables[1]
# Replace every 'TH' with 'TD' in the 'path' key
for item in table:
    item['path'] = item['path'].replace('TH', 'TD')

# only need to look at the first row
first_row = [el for el in table if "TR[2]/" in el['path']]
    # Function to decide if table has 2 indexes

def get_table_dim(table):
    # Regular expressions to find row and column indices
    row_regex = re.compile(r'/TR\[(\d+)\]')
    col_regex = re.compile(r'/TD\[(\d+)\]')

    max_row = 0
    max_col = 0

    for entry in table:
        path = entry['path']  # Extract path from the dictionary
        
        # Extract row and column indices
        row_match = row_regex.search(path)
        col_match = col_regex.search(path)
        
        # Update max row index
        if row_match:
            row_index = int(row_match.group(1))
            max_row = max(max_row, row_index)
        
        # Update max column index
        if col_match:
            col_index = int(col_match.group(1))
            max_col = max(max_col, col_index)
        elif '/TD/' in path and not col_match:  # Check for default first column
            max_col = max(max_col, 1)  # Ensure first column is counted
    return (max_row, max_col)    

# Function to check if the API could not identify the index of the table
def is_unidentified(table):
    table_dim = get_table_dim(table)
    min_headers = min(table_dim)
    header_count = len([el for el in table if "TH" in el['path']])
    if header_count < min_headers:
        return True
    return False

# Function to transform the unidentified table by determining its index using LLM
def transform_unidentified_table(table):
    # Function to get the first 3 rows 
    def get_first_three_rows(table):
            # Regular expression to extract row index
            row_regex = re.compile(r'/TR\[(\d+)\]')
            
            # List to hold the first three rows table
            rows = []
            
            for entry in table:
                path = entry['path']
                # Find row index
                row_match = row_regex.search(path)
                
                if row_match:
                    row_index = int(row_match.group(1))
                    # Check if the row index is within the first three rows
                    if 1 <= row_index <= 3:
                        rows.append(entry)
                elif '/TR/' in path:  # Check for paths that imply the first row implicitly
                    rows.append(entry)
        
            table_first_3_rows = [] # This will be a 2D list
            table_rows = set([re.search(r'Table(?:\[\d+\])?/TR(?:\[\d+\])?', item['path']).group() for item in table if re.search(r'Table(?:\[\d+\])?/TR(?:\[\d+\])?', item['path'])])
            uniq_rows = sorted([row_name.split("/")[-1] for row_name in list(table_rows)[:3]])
            uniq_rows[0] += "/"


            for i in range(len(uniq_rows)):
                row = [el for el in table if uniq_rows[i] in el['path']]
                unique_tds = set()
                for item in row:
                            path_parts = item['path'].split('/')
                            for part in path_parts:
                                if 'TD' in part:
                                    unique_tds.add(part)
                # Convert the set to a list and sort it for consistent output
                unique_tds_list = sorted(list(unique_tds))
                        
                sections_of_row = []
                for td in unique_tds_list:
                            td_section = ""
                            for i in range(len(row)):
                                if i ==0:
                                    td += "/"
                                if td in row[i]['path']:
                                    td_section += row[i]['text'].strip()

                            sections_of_row.append(td_section)
                
                # print(sections_of_row)
                table_first_3_rows.append(sections_of_row)

            # account for case where there are 2 index columns:
            min_dim = max(list(map(lambda x: len(x), table_first_3_rows)))
            if len(table_first_3_rows[0]) < min_dim:
                table_first_3_rows[0].insert(0, "empty")
            return table_first_3_rows

    # Function to get the first 3 rows of the table into a string to parse to the LLM 
    def get_table_check_string(table_first_3_rows):
        table_str = ''
        for i in range(len(table_first_3_rows)):
            table_str += f'''Row{i+1}: {" | ".join(table_first_3_rows[i])}\n'''
        return table_str

    # Function to clean the output if it decides to explain its choice.
    def clean_llama_output_if_string(input_str):
        """
        Extracts a dictionary-like substring from the given input string.
        
        Args:
        input_str (str): The input string containing the dictionary-like substring.
        
        Returns:
        dict: The extracted dictionary as a Python dictionary.
        """
        # Use a regular expression to find the dictionary-like substring
        match = re.search(r'\{.*?\}', input_str)
        
        if match:
            dict_str = match.group()
            try:
                # Safely evaluate the dictionary string
                extracted_dict = eval(dict_str)
                if isinstance(extracted_dict, dict):
                    return extracted_dict
            except (SyntaxError, NameError):
                pass
        
        return None

    # Function to decide the indexing of the table
    def eval_table_index_llama(table_str):
        class Header(BaseModel):
            index: int = Field(description=(
                '''Index of the table indicating the type of indexing:\n\
                    0 - No indexes\n\
                    1 - Indexed by the first column\n\
                    2 - Indexed by the first row\n\
                    3 - Indexed by both the first row and the first column'''))

        parser = JsonOutputParser(pydantic_object=Header)

        chat = ChatGroq(temperature=0, model_name="llama3-8b-8192")
        
        template = '''You will assist me in deciding, based on the first 3 entries of a table, whether the first row or the first colum should be the header, or if both the first column and first row are headers
    Table:
    {table}

    Determine the indexing type of the table and output:
    - 0 if the table is indexed by the first column.
    - 1 if the table is indexed by the first row.
    - 2 if the table is indexed by both the first row and the first column.   
                
    {format_instructions}'''
        prompt = PromptTemplate(
            template=template,
            input_variables=["table"],
            partial_variables={"format_instructions": parser.get_format_instructions()},
        )
            
        chain = prompt | chat | parser
        return chain.invoke({"table": table_str})
    
    table_first_3_rows = get_first_three_rows(table)
    table_str = get_table_check_string(table_first_3_rows)
    header_output = eval_table_index_llama(table_str)
    if isinstance(header_output, str):
        header_output = clean_llama_output_if_string(header_output)
    header_index = header_output['index']
    # Now need diff cases of how to handle the table if the model outputs, 0,1, or 2
        # where 0 is the first col, 1 is second col, 2 is for both cols are headers
    pass

if is_unidentified(table):
    processed_unidentified_table = transform_unidentified_table(table)
    pass


## Old functions 

In [4]:

# Get the extracted data from the extractor
def get_extracted_data(extracted_data):
    if 'structuredData.json' in extracted_data:
        json_data = json.loads(extracted_data['structuredData.json'])
    return json_data



# This function converts tables to strings to be able to be processed by LLMs. 
def get_and_save_table_strings(extractor, table_output_directory):
    
    # Function to convert each row in a raw unprocessed table (ie index of table not decided) into a string
    def get_raw_table_string(df):
        table_str = ""
        for i in range(2):
            if i ==1:
                table_str += f"Row {i}: {df.iloc[i].values.tolist()}"  
            else:
                table_str += f"Row {i}: {df.iloc[i].values.tolist()}\n"
        return table_str
    
    # Function to decide if header is first row or first column
    def evaluate_table_index_llama(table_str):
        class Header(BaseModel):
            index: int = Field(description="Header of the table, 0 for first row as the header, 1 for first column as the header")
            
        parser = JsonOutputParser(pydantic_object=Header)

        chat = ChatGroq(temperature=0, model_name="llama3-8b-8192")
        
        template = '''You will assist me in deciding, based on the first 2 entries of a table, whether the first row or the first colum should be the header. 
                You are to output an int, 0 or 1. Where 0 if the first row is header, and 1 if the first column is the header.
                Follow the format instructions carefully.
                Table:
                {table}
                
                {format_instructions}
                '''
        prompt = PromptTemplate(
            template=template,
            input_variables=["table"],
            partial_variables={"format_instructions": parser.get_format_instructions()},
        )
        chain = prompt | chat | parser
        return chain.invoke({"table": table_str})

    # Tables need procecssing when extraced from BytesIO
    def clean_table_values(x):
        if isinstance(x, str):
            return x.replace('_x000D_', '').strip()
        return x
    
    # Code adapted from a medium blog on how to represent rows of tables in strings
    def convert_table_to_string(df):
        for _, row in df.iterrows():
            row_str = ""
            for col in df.columns:
                sentences = re.split(r'(?<=\.)\s*', row[col])
                row_sentence = ""
                for i in range(len(sentences)):
                    row_sentence += sentences[i] + "\n"
                row_str += f"{col}: {row_sentence}, "
            formatted = row_str[:-2]
        return formatted

    os.makedirs(table_output_directory, exist_ok=True)
    
    # The literal extraction of the file itself
    excel_files = {k: v for k, v in extractor.extracted_data.items() if k.endswith('.xlsx')}
    
    table_dataframes = {}

    num_tables =0
    for _, content in excel_files.items():
        excel_stream = BytesIO(content)
        df = pd.read_excel(excel_stream, header=None)
        
        df = df.applymap(clean_table_values)
        # # Uncomment bottom code to ensure that Groq decides table header index 
        
        # df_str = get_raw_table_string(df) 
        # dic = evaluate_table_index_llama(df_str)
        # header_index = dic['index']
        
        # Uncomment this if u uncomment the code above
        header_index = 1
        
        # If header_index is non zero
        if header_index == 1:
            df = pd.read_excel(excel_stream, header=None)
            df = df.applymap(clean_table_values)
            df = df.T
            # Set the first row as the new header
            new_header = df.iloc[0]  # Take the first row for the header
            df = df[1:]  # Take the data less the header row
            df.columns = new_header  # Set the header row as the df header
            # Optionally, reset index if necessary
            df.reset_index(drop=True, inplace=True)
        else:
            df = pd.read_excel(excel_stream, header=0)
        
        output_file_path = os.path.join(table_output_directory, f"table_{num_tables}.csv")

        # Writing the table to the corresponding csv file. 
        df.to_csv(output_file_path, index=False)
            
        table_str = convert_table_to_string(df)
        table_dataframes[num_tables] = table_str
        num_tables += 1
        
    return table_dataframes



# Function to initialise a flexible text splitter
def initialise_text_splitter(chunk_size, chunk_overlap):
    separator_list = ["\n\n", "\n", ". ", "!", "?", ",", " ", "", ")", "("]
    try:
        text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=chunk_size,
                chunk_overlap=chunk_overlap,
                length_function=len,
                separators=separator_list)
    except Exception as e:
        logging.error(f"Failed to initialize text splitter: {e}")
        return None
    return text_splitter



# Function to obtain text chunks using the text splitter
def get_text_chunks(file_path, json_data, text_splitter):
    if 'elements' not in json_data:
        logging.error("Missing 'elements' key in json_data")
        raise ValueError("Missing 'elements' key in json_data")

    if not text_splitter:
        logging.error("Text splitter not initialised properly. ")
        sys.exit()  
        
    file_name = file_path.split("/")[-1]

    # Chunks are split by pages here
    page_text = ""
    start_page = 0
    all_chunks = []
    
    list_label = ""
    for i in range(len(json_data['elements'])):
        try:
            current_page = json_data['elements'][i]['Page']
        except KeyError:
            logging.warning(f"Missing 'Page' key in element at index {i}")
            continue

        try:
            if current_page > start_page:
                # Update the new page number
                start_page = current_page               
                # Generate the chunks for the previous page
                separated_list = text_splitter.split_text(page_text)
                for chunk in separated_list:
                    if chunk not in [". ", "."]:  # Simplified condition
                        all_chunks.append({'ElementType': 'Text', 'file_name': file_name, 'Page': current_page, 'Text': chunk})
                # Update the string of text 
                page_text = ""
                list_label = ""
            else:
                if 'Text' in json_data['elements'][i]:  # Check if Text is not empty
                    if json_data['elements'][i]['Path'].endswith("Lbl") and not json_data['elements'][i]['Path'].startswith("//Document/Table"):
                        list_label = json_data['elements'][i]['Text']
                    else:
                        if list_label:
                            page_text += list_label + json_data['elements'][i]['Text']
                            list_label = ""  # Reset list label to empty string
                        else:
                            page_text += json_data['elements'][i]['Text'] + "\n"
        except KeyError as e:
            logging.warning(f"Key error in json_data['elements'][i] processing at index {i}: {e}")
    
    
    # Processing the last page of the text
    if page_text:
        separated_list = text_splitter.split_text(page_text)
        for chunk in separated_list:
            if chunk not in [". ", "."]:
                all_chunks.append({'ElementType': 'Text', 'file_name': file_name, 'Page': current_page, 'Text': chunk})

    return all_chunks


# Further enhancement to include the tables with the metadata so that it can be parsed to the 
    # function that takes in the tables + metadata for embeddings generation
def get_table_strings_with_metadata(table_dataframes, json_data):
    
    # Function to obtain the page number of each table
    def get_table_pages(json_data):
        table_file_pages = {}
        # Obtaining the table metadata
        for i in range(len(json_data['elements'])):
            try:
                file_paths = json_data['elements'][i].get('filePaths')
                if file_paths:
                    page = json_data['elements'][i].get('Page', 'Unknown')
                    match = re.search(r'\d+', file_paths[0])
                    table_index = match.group(0)
                    table_file_pages[int(table_index)] = {"Page": page}
            except Exception as e:
                logging.error(f"Error processing file paths at index {i}: {e}")
        return table_file_pages
    
    table_file_pages = get_table_pages(json_data)
    meta_table_batch = []
    table_dfs = []
    for table_index,_ in table_dataframes.items():
        dic = {}
        dic['ElementType'] = 'Table'
        dic['Page'] = table_file_pages[table_index]['Page']
        dic['Table'] = table_dataframes[table_index]
        table_dfs.append(dic)

        #  Obtain metadata for sparse embeddings
        meta_table_batch.append(f"ElementType 'Table', Page {table_file_pages[table_index]['Page']}, {table_dataframes[table_index]}")
    return table_dfs, meta_table_batch

In [4]:
file_path = '../PDF/HSI1000-chapter1.pdf'
extractor, unique_id = ExtractTextTableInfoFromPDF.create_with_unique_id(file_path)
print("\nUnique ID:", unique_id)
table_output_directory = f"../data/{unique_id}"
extracted_data = extractor.extracted_data
pdf_data = get_extracted_data(extracted_data)
table_dataframes = get_and_save_table_strings(extractor, table_output_directory)

# Use some form of evaluator to decide chunk size?
text_splitter = initialise_text_splitter(300, 50)

# Get out important information
text_chunks = get_text_chunks(file_path, pdf_data, text_splitter)
# table_dfs, meta_table_batch= get_table_strings_with_metadata(table_dataframes, pdf_data)

2024-07-02 18:38:10,933 - INFO - Started uploading asset
2024-07-02 18:38:15,314 - INFO - Finished uploading asset
2024-07-02 18:38:15,316 - INFO - Started submitting EXTRACT_PDF job
2024-07-02 18:38:16,553 - INFO - Started getting job result
2024-07-02 18:38:27,172 - INFO - Finished polling for status
2024-07-02 18:38:27,176 - INFO - Finished getting job result
2024-07-02 18:38:27,178 - INFO - Started getting content
2024-07-02 18:38:27,486 - INFO - Finished getting content



Unique ID: 0e4dcde5-fe65-4391-ade1-d1389e41d635
